In [1]:
#!pip install pyarrow
#!pip install pandas
#!pip install fastparquet

In [2]:
import pandas as pd

In [5]:
#!pip install seaborn
#!pip install matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
from sklearn.linear_model import LinearRegression

In [28]:
def read_dataframe(url):
    df = pd.read_parquet(url)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    return df

In [29]:
df_train = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-02.parquet")

In [36]:
#df_train['PU_DO'] = df_train['PULocationID'] + '_' +df_train['DOLocationID']
#df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [32]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
train_dicts = df_train[categorical+numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

from sklearn.metrics import root_mean_squared_error 

print(root_mean_squared_error(y_pred, y_val))

9.018920764427403


In [38]:
import pickle
import os 

output_path = 'models/model.bin'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'wb') as f:
    pickle.dump((dv, model), f)